In [1]:
import pandas as pd
import numpy as np
##
from sklearn import tree
##
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
PATH = 'train.csv'

In [3]:
data = pd.read_csv(PATH)#.replace(np.nan,np.mean(data.Age.values))
columns=data.columns.values

In [4]:
columns

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

# Data Preprocessing

In [5]:
# Mapping Genders
sex = data['Sex'].replace(['female','male'],[0,1])
data.Sex = sex

In [6]:
#cabin = data.Cabin.replace(['C','E'],[1,2])
age = data.Age.values
age = age[~np.isnan(age)]
data['Age'].replace(np.nan,np.mean(age),inplace = True)
fare = data.Fare.values
fare = fare[~np.isnan(fare)]
data['Fare'].replace(np.nan,np.mean(fare), inplace = True)

In [7]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",1,29.699118,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,C


In [8]:
data.Pclass = data.Pclass.astype(int)
data.Sex = data.Sex.astype(int)
data.Age = data.Age.astype(float)
data.SibSp = data.SibSp.astype(int)
data.Parch = data.Parch.astype(int)
data.Fare = data.Fare.astype(float)

# Drop unnecessary columns & Remove Label
Dropped:
- PassengerId
- Name
- Ticket
- Parch
- Cabin
- Embarked

In [ ]:
# Remove Label
y=data['Survived'].values
temp_data=data.drop(['Survived'],axis=1)

In [9]:
# Drop unnecessary colum
###trial###
temp_data=temp_data.drop(['PassengerId', 'Name','Ticket','Parch','Cabin','Embarked'],axis=1)
###
'''
age = temp_data.Age.values
age = (age-np.mean(age))/np.var(age)
fare = temp_data.Fare.values
fare = (fare-np.mean(fare))/np.var(fare)
temp_data.Age = age
temp_data.Fare = fare
'''

'\nage = temp_data.Age.values\nage = (age-np.mean(age))/np.var(age)\nfare = temp_data.Fare.values\nfare = (fare-np.mean(fare))/np.var(fare)\ntemp_data.Age = age\ntemp_data.Fare = fare\n'

In [10]:
x=temp_data.values
columns = temp_data.columns.values

# Test/Train split

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=0)

y_test=y_test[:,np.newaxis]

# Logistic Regression

In [12]:

mms=StandardScaler()#正規化
x_train_std=mms.fit_transform(x_train)
x_test_std=mms.fit_transform(x_test)


In [13]:
#'''
from sklearn.linear_model import LogisticRegression
#LogisticRegression(penalty='l1')

lr=LogisticRegression(penalty='l1', C=0.1)#l1+0.1的學習速率


lr=lr.fit(x_train_std, y_train)# 擬合模型

print(lr.score(x_train_std, y_train))#training的準確度

print(lr.score(x_test_std, y_test))#test的準確度


print(lr.coef_)#抓取所有的權重

for i in range(0,5):
    print(columns[i], lr.coef_[0][i])#print 所有權種
    
csv = pd.DataFrame(lr.predict(x_test_std)).to_csv('predict.csv', index = False)
#'''


D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.797752808988764
0.7947761194029851
[[-0.70892122 -1.13066592 -0.30434738 -0.25333464  0.01496506]]
Pclass -0.7089212223050226
Sex -1.1306659169506312
Age -0.30434737508520593
SibSp -0.25333463916090243
Fare 0.014965058058359028


# Decision Tree

In [14]:
clf = tree.DecisionTreeClassifier(criterion  = 'entropy', max_depth = 3)
clf = clf.fit(x_train_std,y_train)
print(clf.score(x_train_std, y_train))
print(clf.score(x_test_std, y_test))

0.8362760834670947
0.8208955223880597


# Data Prediction

In [15]:
PATH = 'test.csv'
df_test = pd.read_csv(PATH)

In [16]:
sex = df_test['Sex'].replace(['female','male'],[0,1])
df_test.Sex = sex

age = df_test.Age.values
age = age[~np.isnan(age)]
df_test['Age'].replace(np.nan,np.mean(age),inplace = True)

fare = df_test.Fare.values
fare = fare[~np.isnan(fare)]
df_test['Fare'].replace(np.nan,np.mean(fare), inplace = True)

temp_test = df_test.drop(['PassengerId','Name','Ticket','Parch','Cabin','Embarked'],axis=1)


In [17]:
test = temp_test.values 
test_true = mms.fit_transform(test)
csv = pd.DataFrame(clf.predict(test_true), df_test.PassengerId.values).to_csv('predict.csv', index = True)
